# Amazon Reviews Analysis
---
<b>By David Penny<b>

In [246]:
# Let's start by importing our (very) favorite libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [301]:
df_numerical = pd.read_csv('df_numerical.csv').drop(columns='Unnamed: 0')

In [302]:
display(df_numerical.head(1))
print(df_numerical.shape)

,falsified,rating,verified_purchase,sentiment,num_words_in_text,num_stopwords,num_words_in_text_no_stop,num_unique_words,mean_word_len,num_chars,num_punctuations,num_scentences_in_text,flesch_ease,flesch_kincaid_grade,automated_readability_index,overall_readability_index,total_sentiment,average_review_sentiment
0,1,4,0,1,23,11,12,21,4.086957,116,3,2,102.1,1.9,3.6,5.0,6.0,3.0


(19989, 18)


In [324]:
df_downsampled = pd.read_csv('df_downsampled.csv').drop(columns='Unnamed: 0')

In [325]:
display(df_downsampled.head(1))
print(df_downsampled.shape)

,falsified,verified_purchase,sentiment,num_punctuations,flesch_kincaid_grade,overall_readability_index,total_sentiment,average_review_sentiment
0,1,1,1,5,10.5,11.0,16.0,8.0


(9104, 8)


In [326]:
df_downsampled.columns

Index(['falsified', 'verified_purchase', 'sentiment', 'num_punctuations',
       'flesch_kincaid_grade', 'overall_readability_index', 'total_sentiment',
       'average_review_sentiment'],
      dtype='object')

In [327]:
# Here we choose which version of the df we will use
#dataframe = df_numerical
dataframe = df_downsampled

- Let's get a baseline to see how a dummy classifier would score.

In [329]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Separate input features and target
y = df_final['falsified']
X = df_final.drop(columns='falsified', axis=1)

# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)

# DummyClassifier to predict only target 0
dummy = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)
dummy_pred = dummy.predict(X_test)

# checking unique labels
print('Unique predicted labels: ', (np.unique(dummy_pred)))

# checking accuracy
print('Test score: ', accuracy_score(y_test, dummy_pred))

Unique predicted labels:  [1]
Test score:  0.49615975422427033


### 10. Machine Learning

In [330]:
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

In [331]:
y = df_final['falsified']
X = df_final.drop(columns='falsified')

print(y.shape)
print(X.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

NN_model = MLPClassifier(hidden_layer_sizes=(128,128,128), solver='lbfgs', max_iter=10000)
NN_model.fit(X,y);

print(f'The test accuracy is: {NN_model.score(X,y):0.3f}')

(20830,)
(20830, 23)


ValueError: could not convert string to float: 'Sports'

In [277]:
import tensorflow as tf
import datetime

In [278]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [332]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

5826 train examples
1457 validation examples
1821 test examples


In [333]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('falsified')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [334]:
batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [335]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['total_sentiment'])
  print('A batch of targets:', label_batch )

Every feature: ['verified_purchase', 'sentiment', 'num_punctuations', 'flesch_kincaid_grade', 'overall_readability_index', 'total_sentiment', 'average_review_sentiment']
A batch of ages: tf.Tensor([12. 18. 15.  9.  4.], shape=(5,), dtype=float64)
A batch of targets: tf.Tensor([0 1 0 0 0], shape=(5,), dtype=int64)


In [336]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('falsified')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [337]:
batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [338]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['total_sentiment'])
  print('A batch of targets:', label_batch )

Every feature: ['verified_purchase', 'sentiment', 'num_punctuations', 'flesch_kincaid_grade', 'overall_readability_index', 'total_sentiment', 'average_review_sentiment']
A batch of ages: tf.Tensor([12. 11. 10.  8.  7.], shape=(5,), dtype=float64)
A batch of targets: tf.Tensor([1 1 0 0 1], shape=(5,), dtype=int64)


In [339]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]

In [340]:
# # A utility method to create a feature column
# # and to transform a batch of data
# def demo(feature_column):
#   feature_layer = layers.DenseFeatures(feature_column)
#   print(feature_layer(example_batch).numpy())

In [341]:
# total_sentiment = feature_column.numeric_column("total_sentiment")
# demo(total_sentiment)

In [342]:
# total_sentiment_buckets = feature_column.bucketized_column(total_sentiment, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
# demo(total_sentiment_buckets)

In [343]:
# thal = feature_column.categorical_column_with_vocabulary_list(
#       'thal', ['fixed', 'normal', 'reversible'])

# thal_one_hot = feature_column.indicator_column(thal)
# demo(thal_one_hot)

In [344]:
# # Notice the input to the embedding column is the categorical column
# # we previously created
# thal_embedding = feature_column.embedding_column(thal, dimension=8)
# demo(thal_embedding)

In [345]:
# thal_hashed = feature_column.categorical_column_with_hash_bucket(
#       'thal', hash_bucket_size=1000)
# demo(feature_column.indicator_column(thal_hashed))

In [346]:
# crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
# demo(feature_column.indicator_column(crossed_feature))

In [347]:
feature_columns = []

# # numeric cols
# for header in ['rating',
#                'verified_purchase',
#                'sentiment',
#                'num_words_in_text',
#                'num_stopwords',
#                'num_words_in_text_no_stop', 
#                'num_unique_words', 
#                'mean_word_len',
#                'num_chars',
#                'num_punctuations',
#                'num_scentences_in_text', 
#                'flesch_ease', 
#                'flesch_kincaid_grade',
#                'automated_readability_index',
#                'overall_readability_index', 
#                'total_sentiment',
#                'average_review_sentiment']:
#   feature_columns.append(feature_column.numeric_column(header))

# DOWNSAMPLED numeric cols
for header in ['verified_purchase', 'sentiment', 'num_punctuations',
       'flesch_kincaid_grade', 'overall_readability_index', 'total_sentiment',
       'average_review_sentiment']:
  feature_columns.append(feature_column.numeric_column(header))

# # bucketized cols
# age_buckets = feature_column.bucketized_column(total_sentiment, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
# feature_columns.append(age_buckets)

# # indicator cols
# thal = feature_column.categorical_column_with_vocabulary_list(
#       'thal', ['fixed', 'normal', 'reversible'])
# thal_one_hot = feature_column.indicator_column(thal)
# feature_columns.append(thal_one_hot)

# # embedding cols
# thal_embedding = feature_column.embedding_column(thal, dimension=8)
# feature_columns.append(thal_embedding)

# # crossed cols
# crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
# crossed_feature = feature_column.indicator_column(crossed_feature)
# feature_columns.append(crossed_feature)

In [348]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [349]:
batch_size = 50
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [350]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(4, activation='sigmoid'),
  layers.Dense(128, activation='sigmoid'),
  layers.Dense(128, activation='sigmoid'),
  layers.Dense(128, activation='sigmoid'),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(train_ds,
          validation_data=val_ds,
          callbacks=[tensorboard_callback],
          epochs=20)

Epoch 1/20

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

117/117 [==============================] - 1s 4ms/step - loss: 0.6969 - accuracy: 0.4882 - val_loss: 0.6918 - val_accuracy: 0.4804
Epoch 2/20
117/117 [==============================] - 0s 3ms/step - loss: 0.6941 - accuracy: 0.4882 - val_loss: 0.6914 - val_accuracy: 0.4804
Epoch 3/20
117/117 [==============================] - 0s 3ms/step - loss: 0.6880 - accuracy: 0.4900 - val_loss: 0.6888 - val_accuracy: 0.4804
Epoch 4/20
117/117 [==============================] - 0s 3ms/step - loss: 0.6557 -

In [298]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

84/84 [==============================] - 0s 2ms/step - loss: 0.4635 - accuracy: 0.7895
Accuracy 0.789486289024353


Next steps
The best way to learn more about classifying structured data is to try it yourself. We suggest finding another dataset to work with, and training a model to classify it using code similar to the above. To improve accuracy, think carefully about which features to include in your model, and how they should be represented.

In [299]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [300]:
%tensorboard --logdir logs/fit

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "/Users/davepenny/opt/anaconda3/envs/deeplearning/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/Users/davepenny/opt/anaconda3/envs/deeplearning/lib/python3.6/site-packages/tensorboard/main.py", line 65, in run_main
    default.get_plugins() + default.get_dynamic_plugins(),
  File "/Users/davepenny/opt/anaconda3/envs/deeplearning/lib/python3.6/site-packages/tensorboard/default.py", line 125, in get_dynamic_plugins
    "tensorboard_plugins"
  File "/Users/davepenny/opt/anaconda3/envs/deeplearning/lib/python3.6/site-packages/tensorboard/default.py", line 124, in <listcomp>
    for entry_point in pkg_resources.iter_entry_points(
  File "/Users/davepenny/opt/anaconda3/envs/deeplearning/lib/python3.6/site-packages/pkg_resources/__init__.py", line 2460, in load
    self.require(*args, **kwargs)
  File "/Users/davepenny/opt/anaconda3/envs/deeplearning/lib

In [276]:
# Clear any logs from previous runs
!rm -rf ./logs/ 